<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/step2-tilt_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tilt search
--- 

Here we will test a range of 25 tilts evenly spaced from [0.05,2] at .45 contrast in each trained network and pick those that produce ~60%, ~70%, and ~80% correct to match human behavioral performance. Then, we will test these 3 tilts with contrasts of 1, .45, and 0.3, respectively.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 16 18:33:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## 1: Setup the model
Choose the model we will be using to search for qualifying tilts. The models were created using the `step1` Jupyter notebook. 
Run the cell to download a zip file from OSF then extract its contents into the newly created directory.

vanilla gabor model: `content/van_gabor/`

In [3]:
# @title Download vanilla Gabor model

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping `vanilla model trained on Gabors`...")
name = 'van_gabor_model.zip'
fname = f"{name}.zip"
url = f"https://osf.io/3wqsf/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `vanilla model trained on Gabors`...
Download completed.


#### Load the model
Next, we load the model using Tensorflow

In [4]:
import tensorflow as tf 
import pandas as pd
tf.random.set_seed(42) #set random seed for reproducibility
model = tf.keras.models.load_model('van_gabor') 
#model = tf.keras.models.load_model('van_gabor') 
model.summary() #verify architecture

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 160, 160, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 158, 158, 160)     1600      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 79, 79, 160)      0         
 )                                                               
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 39, 39, 160)      0         
 2D)                                                             
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 80)        115280    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 18, 18, 80)       0

## 2: Download & load datasets to test model with
Download the test datasets from OSF and extract the contents into the newly created directory: `content/datasets/`

In [5]:
# @title Download datasets

print("Start downloading and unzipping 25 test datasets...")
name = 'tilt_0_05-2_contrast_0_45'
fname = f"{name}.zip"
url = f"https://osf.io/yavbx/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall("datasets") #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping 25 test datasets...
Download completed.


Load all 25 sets and use prefetch to streamline image loading. Contrast is 0.45 and the 25 tilts are: 

    [0.0500, 0.1313, 0.2125, 0.2938, 0.3750, 0.4562, 0.5375, 0.6188, 0.7000, 0.7813, 0.8625, 0.9438, 1.0250, 1.1062, 1.1875, 1.2687, 1.3500, 1.4312, 1.5125, 1.5938, 1.6750, 1.7563, 1.8375, 1.9187, 2.0000]

In [6]:
# @title Load datasets into tensorflow

from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32 
IMG_SIZE = (160, 160) #forces a resize from 169x169 
AUTOTUNE = tf.data.AUTOTUNE #prompts the tf.data runtime to tune the value dynamically at runtime
def model_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE):
    curr_dir = os.getcwd() 
    set1_dir = os.path.join(curr_dir, 'datasets/s1')
    set2_dir = os.path.join(curr_dir, 'datasets/s2')
    set3_dir = os.path.join(curr_dir, 'datasets/s3')
    set4_dir = os.path.join(curr_dir, 'datasets/s4')
    set5_dir = os.path.join(curr_dir, 'datasets/s5')
    set6_dir = os.path.join(curr_dir, 'datasets/s6')
    set7_dir = os.path.join(curr_dir, 'datasets/s7')
    set8_dir = os.path.join(curr_dir, 'datasets/s8')
    set9_dir = os.path.join(curr_dir, 'datasets/s9')
    set10_dir = os.path.join(curr_dir, 'datasets/s10')
    set11_dir = os.path.join(curr_dir, 'datasets/s11')
    set12_dir = os.path.join(curr_dir, 'datasets/s12')
    set13_dir = os.path.join(curr_dir, 'datasets/s13')
    set14_dir = os.path.join(curr_dir, 'datasets/s14')
    set15_dir = os.path.join(curr_dir, 'datasets/s15')
    set16_dir = os.path.join(curr_dir, 'datasets/s16')
    set17_dir = os.path.join(curr_dir, 'datasets/s17')
    set18_dir = os.path.join(curr_dir, 'datasets/s18')
    set19_dir = os.path.join(curr_dir, 'datasets/s19')
    set20_dir = os.path.join(curr_dir, 'datasets/s20')
    set21_dir = os.path.join(curr_dir, 'datasets/s21')
    set22_dir = os.path.join(curr_dir, 'datasets/s22')
    set23_dir = os.path.join(curr_dir, 'datasets/s23')
    set24_dir = os.path.join(curr_dir, 'datasets/s24')
    set25_dir = os.path.join(curr_dir, 'datasets/s25')
    set1 = image_dataset_from_directory(set1_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale') #2000 images 2 classes
    set2 = image_dataset_from_directory(set2_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set3 = image_dataset_from_directory(set3_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set4 = image_dataset_from_directory(set4_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set5 = image_dataset_from_directory(set5_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set6 = image_dataset_from_directory(set6_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set7 = image_dataset_from_directory(set7_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale') 
    set8 = image_dataset_from_directory(set8_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set9 = image_dataset_from_directory(set9_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set10 = image_dataset_from_directory(set10_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set11 = image_dataset_from_directory(set11_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set12 = image_dataset_from_directory(set12_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set13 = image_dataset_from_directory(set13_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale') 
    set14 = image_dataset_from_directory(set14_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set15 = image_dataset_from_directory(set15_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set16 = image_dataset_from_directory(set16_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set17 = image_dataset_from_directory(set17_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set18 = image_dataset_from_directory(set18_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set19 = image_dataset_from_directory(set19_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set20 = image_dataset_from_directory(set20_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set21 = image_dataset_from_directory(set21_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale') 
    set22 = image_dataset_from_directory(set22_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set23 = image_dataset_from_directory(set23_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set24 = image_dataset_from_directory(set24_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    set25 = image_dataset_from_directory(set25_dir, shuffle=False, batch_size=BATCH_SIZE, image_size=IMG_SIZE, color_mode='grayscale')
    class_names = set1.class_names #extract class names loading function inferred from subdir's
    set1 = set1.prefetch(buffer_size=AUTOTUNE) 
    set2 = set2.prefetch(buffer_size=AUTOTUNE) 
    set3 = set3.prefetch(buffer_size=AUTOTUNE) 
    set4 = set4.prefetch(buffer_size=AUTOTUNE) 
    set5 = set5.prefetch(buffer_size=AUTOTUNE) 
    set6 = set6.prefetch(buffer_size=AUTOTUNE) 
    set7 = set7.prefetch(buffer_size=AUTOTUNE) 
    set8 = set8.prefetch(buffer_size=AUTOTUNE) 
    set9 = set9.prefetch(buffer_size=AUTOTUNE) 
    set10 = set10.prefetch(buffer_size=AUTOTUNE)
    set11 = set11.prefetch(buffer_size=AUTOTUNE)
    set12 = set12.prefetch(buffer_size=AUTOTUNE)
    set13 = set13.prefetch(buffer_size=AUTOTUNE)
    set14 = set14.prefetch(buffer_size=AUTOTUNE)
    set15 = set15.prefetch(buffer_size=AUTOTUNE)
    set16 = set16.prefetch(buffer_size=AUTOTUNE)
    set17 = set17.prefetch(buffer_size=AUTOTUNE)
    set18 = set18.prefetch(buffer_size=AUTOTUNE)
    set19 = set19.prefetch(buffer_size=AUTOTUNE)
    set20 = set20.prefetch(buffer_size=AUTOTUNE)
    set21 = set21.prefetch(buffer_size=AUTOTUNE)
    set22 = set22.prefetch(buffer_size=AUTOTUNE)
    set23 = set23.prefetch(buffer_size=AUTOTUNE)
    set24 = set24.prefetch(buffer_size=AUTOTUNE)
    set25 = set25.prefetch(buffer_size=AUTOTUNE)
    return set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16,set17,set18,set19,set20,set21,set22,set23,set24,set25,class_names

set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16,set17,set18,set19,set20,set21,set22,set23,set24,set25,class_names = model_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE)

Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files belonging to 2 classes.
Found 4000 files

##3: Generate accuracy
First, we can define a function for generating accuracy from a dataset processed by a particular model.

In [7]:
from scipy.special import expit #import sigmoid func
import numpy as np
def get_acc(dataset, model):
    all_pred=tf.zeros([], tf.float64) #initialize array to hold all prediction logits (single element)
    all_labels=tf.zeros([], tf.float64) #initialize array to hold all actual labels (single element)
    for image_batch, label_batch in dataset.as_numpy_iterator():
        predictions = model.predict_on_batch(image_batch).flatten() #run batch through model and return logits
        all_pred = tf.experimental.numpy.append(all_pred, predictions)
        all_labels = tf.experimental.numpy.append(all_labels, label_batch)
    #tf.size(all_pred) #1335 elements, 1334 images + 1 placeholder 0 at beginning
    all_pred = all_pred[1:] #remove placeholder 0 at beginning
    all_labels = all_labels[1:]
    all_pred_sig = expit(all_pred) #sigmoid-transform the logits
    all_pred_round = np.where((all_pred_sig < 0.5), 0, 1) #replace predictions with 0 or 1
    all_acc = np.where((all_pred_round == all_labels), 1, 0) #decide whether pred = label
    return all_pred,all_labels,all_acc

Let's test it on the first dataset:

In [8]:
all_pred, all_labels, all_acc = get_acc(set25,model)

In [9]:
print(all_pred.numpy()[1000:1055]) #first five logits, of 1000
print(all_labels.numpy()[1000:1055]) #first five logits, of 1000
print(all_acc[1000:1055]) #first five logits, of 1000

[-6.57117558 -5.89991283 -7.11768579 -5.59540415 -6.88808489 -5.94354773
 -5.91742659 -6.39437056 -5.53728628 -5.87378073 -6.39160252 -6.2557478
 -6.64357901 -7.00559473 -6.68111181 -7.18521404 -5.92584705 -5.58824492
 -7.21365499 -6.1089859  -5.86634445 -6.0357933  -6.69377279 -3.74141431
 -3.58961177 -7.41236162 -5.94856977 -7.62019968 -6.05494833 -5.57611036
 -4.17303658 -5.09070206 -4.71397114 -4.99580956 -5.85829639 -6.68193436
 -6.01877451 -7.05878687 -4.87531662 -4.83418512 -5.01124096 -4.30645084
 -6.88602638 -4.92856789 -6.54918385 -5.85306883 -6.6007328  -5.54126072
 -7.26403379 -4.87128115 -7.01108313 -6.98884773 -3.99233723 -6.62349892
 -5.73960066]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [10]:
all_acc.mean() #mean stays static for up to 7-8 digits

1.0

This looks accurate. The negative logits were class 0 and positive class 1. All were supposed to be class 0. Hence, the accuracy. 

Now, we can process all datasets and save into a dataframe, `df`.

In [11]:
df = pd.DataFrame(columns=['Tilts','Accuracy']) #initialize dataframe
all_sets = [set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16,set17,set18,set19,set20,set21,set22,set23,set24,set25]
tilts = [0.0500,0.1313,0.2125,0.2938,0.3750,0.4562,0.5375,0.6188,0.7000,0.7813,0.8625,0.9438,1.0250,1.1062,1.1875,1.2687,1.3500,1.4312,1.5125,1.5938,1.6750,1.7563,1.8375,1.9187,2.0000]
idx=0;
for idx, dataset in enumerate(all_sets): #run for all sets:
    all_pred, all_labels, all_acc = get_acc(dataset, model)
    df = pd.concat([df, pd.DataFrame({'Tilts':[tilts[idx]],
                                      'Accuracy':[all_acc.mean()]})], 
                   axis=0, ignore_index=True) #append tilts & accuracies to dataframe

In [12]:
df

,Tilts,Accuracy
0,0.0500,0.57150
1,0.1313,0.67750
2,0.2125,0.78100
3,0.2938,0.84600
4,0.3750,0.90500
5,0.4562,0.93925
6,0.5375,0.97275
7,0.6188,0.98750
8,0.7000,0.99250
9,0.7813,0.99850


Finally, we can save the dataframe as an excel file to disk.

In [13]:
logits_excel_filepath = os.path.join(os.getcwd(), 'tilts_acc-van_gabor.xlsx') #prep path to save to

df.to_excel(logits_excel_filepath, index=False) #save to disk